In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from flask import Flask, request, jsonify

# Load and clean data
df = pd.read_csv("clean_symptom_data.csv", skiprows=1)  # Skip bad header row
df.columns = ['Symptoms', 'Recommended Medicine', 'Diet Recommendation']
df = df.applymap(lambda x: str(x).strip(' "'))  # Strip unwanted quotes and whitespace
df.dropna(inplace=True)

# Features and targets
X = df['Symptoms']
y_medicine = df['Recommended Medicine']
y_diet = df['Diet Recommendation']

# Encode target variables
le_med = LabelEncoder()
le_diet = LabelEncoder()
y_med_enc = le_med.fit_transform(y_medicine)
y_diet_enc = le_diet.fit_transform(y_diet)
Y = np.vstack((y_med_enc, y_diet_enc)).T

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
])

# Train model
pipeline.fit(X_train, Y_train)

# Predict and evaluate
Y_pred = pipeline.predict(X_test)
acc_med = accuracy_score(Y_test[:, 0], Y_pred[:, 0])
acc_diet = accuracy_score(Y_test[:, 1], Y_pred[:, 1])
print(f"✅ Medicine accuracy: {acc_med:.2%}")
print(f"✅ Diet accuracy: {acc_diet:.2%}")

# Prediction function
def predict_recommendations(symptoms_text):
    pred = pipeline.predict([symptoms_text])[0]
    return le_med.inverse_transform([pred[0]])[0], le_diet.inverse_transform([pred[1]])[0]

# Example prediction
symptom_input = "fever, sore throat, cough"
med, diet = predict_recommendations(symptom_input)
print(f"🔮 Prediction -> Medicine: \"{med}\"\n🍽️ Prediction -> Diet: \"{diet}\"")


C:\Users\hp\AppData\Local\Temp\ipykernel_13280\2232841793.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x).strip(' "'))  # Strip unwanted quotes and whitespace


✅ Medicine accuracy: 58.97%
✅ Diet accuracy: 56.41%
🔮 Prediction -> Medicine: "Acetaminophen (paracetamol)"
🍽️ Prediction -> Diet: "Warm broths, soft bland foods, honey"


In [2]:
%pip install flask-cors

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask_cors import CORS 
app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing

def predict_recommendations(symptoms_list):
    # Convert list to comma-separated string
    symptoms_text = ", ".join(symptoms_list).lower()
    pred = pipeline.predict([symptoms_text])[0]
    return le_med.inverse_transform([pred[0]])[0], le_diet.inverse_transform([pred[1]])[0]

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    symptoms_list = data['symptoms']  # Now expects a list
    med, diet = predict_recommendations(symptoms_list)
    return jsonify({
        'medicine': med,
        'diet': diet
    })
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.43.5:5000
Press CTRL+C to quit
127.0.0.1 - - [17/May/2025 18:53:08] "POST /predict HTTP/1.1" 200 -
